In [67]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs


url = requests.get("http://salaryguide.diamondbacklab.com/#/salGuide")
soup = bs(url.content, 'lxml')

In [68]:
from selenium import webdriver

path_to_chromedriver = 'D:\Programming\Scraping\Drivers\chromedriver'
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [69]:
url = 'http://salaryguide.diamondbacklab.com/#/'
browser.get(url)

In [70]:
browser.find_element_by_id('view').click() # click view

In [71]:
browser.find_element_by_id('search').send_keys("CMNS-Computer Science") # only get compsci

In [72]:
row = browser.find_element_by_tag_name('table')

In [73]:
entries = row.text.splitlines()
entries

['EMPLOYEE ▲ DEPARTMENT TITLE SALARY',
 'Adams, Brandi Kristine CMNS-Computer Science Assoc Dir $83,287.26',
 'Agrawala, Ashok K. CMNS-Computer Science Prof $111,685.34',
 'Aloimonos, John CMNS-Computer Science Prof $80,564.98',
 'Atchison, Kate L CMNS-Computer Science Coordinator $53,000.00',
 'Bhattacharjee, Samrat CMNS-Computer Science Prof $191,698.92',
 'Boykin, Regis D. CMNS-Computer Science Accounting Assoc $36,006.20',
 'Carpuat, Marine CMNS-Computer Science Asst Prof $56,107.00',
 'Chen, Ray Sun CMNS-Computer Science Fac Spec $92,058.24',
 'Chick, Brenda K. CMNS-Computer Science Coordinator $56,637.07',
 'Childs, Andrew M CMNS-Computer Science Assoc Prof $67,034.00']

In [74]:
columns = entries[0]
columns

'EMPLOYEE ▲ DEPARTMENT TITLE SALARY'

In [75]:
columns = columns[:8] + columns[21:]
columns = columns.split(' ')
columns

['EMPLOYEE', 'TITLE', 'SALARY']

In [76]:
df = pd.DataFrame(columns = columns)

In [77]:
import re

In [78]:
regex = re.compile('^(.+) CMNS-Computer Science (.+) \$(.+)$')

In [79]:
def get_entries(entries, df):
    for entry in entries[1:]:
        m = regex.match(entry)
        df1 = pd.DataFrame([[m.group(1), m.group(2), m.group(3)]], columns = columns)
        df = df.append(df1)
    return df

In [80]:
df = get_entries(entries, df)

In [81]:
page = row.text
prev_page = 'a'
while(True):
    browser.find_element_by_id('next').click()
    row = browser.find_element_by_tag_name('table')
    prev_page = page
    page = row.text
    if(page == prev_page):
        break
    entries = row.text.splitlines()
    df = get_entries(entries, df)
df

,EMPLOYEE,TITLE,SALARY
0,"Adams, Brandi Kristine",Assoc Dir,"83,287.26"
0,"Agrawala, Ashok K.",Prof,"111,685.34"
0,"Aloimonos, John",Prof,"80,564.98"
0,"Atchison, Kate L",Coordinator,"53,000.00"
0,"Bhattacharjee, Samrat",Prof,"191,698.92"
0,"Boykin, Regis D.",Accounting Assoc,"36,006.20"
0,"Carpuat, Marine",Asst Prof,"56,107.00"
0,"Chen, Ray Sun",Fac Spec,"92,058.24"
0,"Chick, Brenda K.",Coordinator,"56,637.07"
0,"Childs, Andrew M",Assoc Prof,"67,034.00"
